# WSL GPU Setup Guide for CodexContinue

This notebook provides a step-by-step guide to set up GPU support in Windows Subsystem for Linux (WSL) for the CodexContinue project. By following these instructions, you'll be able to use your NVIDIA GeForce RTX 2070 GPU with Ollama in WSL.

## Current Status

Based on the diagnostics, we've identified the following:

1. ✅ You have an NVIDIA GeForce RTX 2070 with 8GB VRAM on your Windows system
2. ✅ You've installed the CUDA repository in WSL
3. ✅ You've installed the NVIDIA Container Toolkit in WSL
4. ❌ The NVIDIA driver for WSL is not installed or not working properly

The key issue is that WSL cannot find your GPU: `WSL environment detected but no adapters were found`

## Step 1: Install NVIDIA Driver for WSL on Windows

The most important step is to install the special NVIDIA driver for WSL on your Windows host system.

1. Open a web browser on Windows (not in WSL)
2. Go to: https://developer.nvidia.com/cuda/wsl
3. Click "Download Now" to download the NVIDIA GPU driver for WSL
4. Run the installer on your Windows system
5. Restart your computer after installation

This driver is different from the regular NVIDIA drivers. It's specifically designed to enable GPU access from WSL.

**Note:** The regular NVIDIA driver you have installed (version 457.51) may be too old to support WSL GPU features properly. The WSL-specific driver installation should update this.

## Step 2: Verify GPU Support in WSL

After installing the driver and restarting your computer, open your WSL terminal and check if the GPU is detected:

```bash
nvidia-smi
```

If properly installed, you should see information about your RTX 2070 GPU.

In [ ]:
# This cell can be run to check if the NVIDIA driver is properly installed in WSL
# You'll need to run it from the Jupyter notebook running in WSL

import subprocess
import sys

def check_nvidia_driver():
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ NVIDIA driver is properly installed and working!")
            print("\nNVIDIA Driver Output:")
            print(result.stdout)
            return True
        else:
            print("❌ NVIDIA driver is not working properly.")
            print("\nError:")
            print(result.stderr)
            return False
    except FileNotFoundError:
        print("❌ NVIDIA driver (nvidia-smi) is not installed or not in PATH.")
        return False

check_nvidia_driver()

## Step 3: Test GPU with Docker

After confirming that `nvidia-smi` works, test if Docker can access the GPU:

```bash
docker run --rm --gpus all nvidia/cuda:11.6.2-base-ubuntu20.04 nvidia-smi
```

If successful, you'll see the same GPU information output from within the container.

In [ ]:
# This cell can be run to check if Docker can access the GPU
# You'll need to run it from the Jupyter notebook running in WSL

import subprocess

def check_docker_gpu():
    try:
        print("Testing GPU access from Docker container...")
        result = subprocess.run(
            ['docker', 'run', '--rm', '--gpus', 'all', 'nvidia/cuda:11.6.2-base-ubuntu20.04', 'nvidia-smi'],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("✅ Docker can access the GPU!")
            print("\nOutput from container:")
            print(result.stdout)
            return True
        else:
            print("❌ Docker cannot access the GPU.")
            print("\nError:")
            print(result.stderr)
            return False
    except Exception as e:
        print(f"❌ Error running Docker command: {e}")
        return False

check_docker_gpu()

## Step 4: Start Ollama with GPU Support

Once Docker can access the GPU, you can start Ollama with GPU acceleration using the provided script:

```bash
./scripts/start-ollama-wsl.sh
```

This script is configured to use the GPU for Ollama in WSL.

In [ ]:
# This cell can be run to start Ollama with GPU support
# You'll need to run it from the Jupyter notebook running in WSL

import subprocess
import os

def start_ollama_with_gpu():
    script_path = os.path.join(os.getcwd(), 'scripts', 'start-ollama-wsl.sh')
    
    if not os.path.exists(script_path):
        print(f"❌ Script not found: {script_path}")
        return False
    
    try:
        print("Starting Ollama with GPU support...")
        # Using Popen to allow the process to run in the background
        process = subprocess.Popen(
            ['bash', script_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )
        
        # Print the first few lines of output
        print("\nInitial output:")
        for i, line in enumerate(process.stdout):
            print(line.strip())
            if i >= 10:  # Only show first 10 lines
                print("...(output continues)...")
                break
                
        print("\n✅ Ollama startup process initiated!")
        print("Check Docker containers to verify it's running: docker ps")
        return True
    except Exception as e:
        print(f"❌ Error starting Ollama: {e}")
        return False

# Uncomment the next line to run this function
# start_ollama_with_gpu()

## Step 5: Run the Full CodexContinue Environment

After confirming Ollama is running with GPU support, you can start the complete development environment:

```bash
./scripts/start-dev-environment.sh
```

This will start all the necessary services for the CodexContinue project.

## Alternative: CPU-Only Mode

If you're having persistent issues with GPU setup, you can run CodexContinue in CPU-only mode:

```bash
docker compose -f docker-compose.yml -f docker-compose.macos.yml up
```

This uses the macOS configuration, which is designed for CPU-only operation.

## Troubleshooting

### Common Issues

1. **WSL NVIDIA Driver Not Detected**: 
   - Make sure you've installed the NVIDIA driver for WSL on Windows (not in WSL)
   - Verify your GPU is compatible with WSL GPU acceleration

2. **Docker GPU Access Issues**:
   - Verify the NVIDIA Container Toolkit is installed: `nvidia-ctk --version`
   - Check Docker configuration: `cat /etc/docker/daemon.json`
   - Make sure Docker has been restarted after configuration

3. **No GPU Found in Container**:
   - Check that `nvidia-smi` works in WSL
   - Verify Docker is configured with GPU access

### Run the Troubleshooting Script

The CodexContinue project provides a troubleshooting script:

```bash
./scripts/troubleshoot-wsl-gpu.sh
```

This script checks your WSL GPU setup and provides guidance for fixing issues.

In [ ]:
# This cell can run the WSL GPU troubleshooting script
# You'll need to run it from the Jupyter notebook running in WSL

import subprocess
import os

def run_troubleshooting_script():
    script_path = os.path.join(os.getcwd(), 'scripts', 'troubleshoot-wsl-gpu.sh')
    
    if not os.path.exists(script_path):
        print(f"❌ Script not found: {script_path}")
        return False
    
    try:
        print("Running WSL GPU troubleshooting script...")
        result = subprocess.run(
            ['bash', script_path],
            capture_output=True, text=True
        )
        
        print("\nTroubleshooting output:")
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ Troubleshooting completed!")
        else:
            print(f"\n⚠️ Troubleshooting script exited with code {result.returncode}")
            if result.stderr:
                print("\nErrors:")
                print(result.stderr)
        
        return result.returncode == 0
    except Exception as e:
        print(f"❌ Error running troubleshooting script: {e}")
        return False

# Uncomment the next line to run the troubleshooting script
# run_troubleshooting_script()

## Using VS Code DevContainer

After setting up GPU support, you can use VS Code's DevContainer feature for development:

1. Open the CodexContinue project in VS Code
2. Click the green icon in the bottom-left corner (or press F1 and search for "Remote-Containers")
3. Select "Reopen in Container"

The DevContainer configuration has been optimized for WSL with the volume mount fixes already implemented.

## Conclusion

By following these steps, you should be able to use your NVIDIA GeForce RTX 2070 GPU with the CodexContinue project in WSL. The GPU acceleration will significantly improve the performance of the Ollama language model.

Remember that the most critical step is installing the NVIDIA driver for WSL on your Windows host system, not in WSL itself.